# Setup

In [ ]:
############################################
##### decide which real machine to use #####
############################################
backend_name = 'ibm_kawasaki'

############################
##### import libraries #####
############################
%matplotlib inline
%load_ext autoreload
%autoreload 2


from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, compiler, circuit, compiler
from pytket.extensions.qiskit import IBMQBackend, AerBackend
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import sys
sys.path.append('../../../aqcel_dev/aqcel_dev')
import aqcel_dev as icepp
sys.path.append('../')
sys.path.append('../qiskit_legacy')
sys.path.append('../qiskit_code')

import QuantumPartonShower as qps
import QuantumPartonShower_ReM as qpsc
import QuantumPartonShower_ReM_2step_hardcode as qps_rhs
import plotting as pl
from PaperPlots import MakeObservables as mo

sim_b = AerBackend()

# ①Original Cirucuit

In [ ]:
N = 1
eps= 0.001
g_1= 2
g_2= 1
initialParticles= ['100'] # f1

shots= 10000

qps0= qps.QuantumPartonShower(N, 1) # for g12= 0
qps1= qps.QuantumPartonShower(N, 1) # for g12= 1

qc0, qubits0= qps0.createCircuit(eps, g_1, g_2, 0, initialParticles)
qc1, qubits1= qps1.createCircuit(eps, g_1, g_2, 1, initialParticles)


# print('Qubits: ' + str(qc0.num_qubits))
# print('Gates: ' + str(qc0.count_ops()))

t1= time.time()
counts0= qps0.simulate('qasm', shots=shots)
t2= time.time()
counts1= qps1.simulate('qasm', shots=shots)
t3= time.time()

# print('\nTime to simulate withg g12= 0: ' + str(t2-t1))
# print('Time to simulate withg g12= 1: ' + str(t3-t2))

# ③AQCEL circuit optimization

In [ ]:
#######################
##### apply aqcel #####
#######################

start_time = time.time()

target_qc = qc0.decompose()
aqcel_results_cc = icepp.pass_manager(target_qc, level=2, backend_tket=sim_b, shots=1024, measure_type='cc').auto_manager()
result_qc = aqcel_results_cc[0]

end_time = time.time()
execution_time = end_time - start_time
print(f"実行時間: {execution_time}秒")

In [ ]:
#############################
##### Before AQCEL(qc1) #####
#############################
print('Before AQCEL')
target_qc.draw('mpl')

In [ ]:
#############################
##### Before AQCEL(qc1) #####
#############################
print('Before AQCEL')
print(f' - Number of gate : {target_qc.count_ops()}')
print(f' - Depth of all : {target_qc.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')


############################
##### After AQCEL(qc2) #####
############################
print('After AQCEL')
print(f' - Number of gate : {result_qc.count_ops()}')
print(f' - Depth of all :{result_qc.depth()}')# 1から始めてmeasurementは含めないで数えてくれる


result_qc.draw('mpl', fold=100)

In [ ]:
###########################
##### Define depth_ls #####
###########################

# これより上のメッセージは無視してください
# depth_lsを同じ値を含む整数を要素に持つリストです。depth_lsの例を出力してください

depth_ls = [1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 240]


# depth_ls = aqcel_results_cc[2]としてください。
depth_ls = aqcel_results_cc[2]

In [ ]:
# AQCELによって取り除かれたCNOTの位置
#############################################
##### Location of CNOT removed by AQCEL #####
#############################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_ls'の中身はCNOTの消された位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸（depth)がnに対してmの高さとなるような棒グラフを出力してください
# グラフのタイトルは'Location of CNOT removed by AQCEL'
# グラフのx軸は'depth'
# グラフのy軸は'Number of CNOTs removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。


import matplotlib.pyplot as plt
from collections import Counter
import numpy as np


# リスト内の各要素の出現回数を数える
counter = Counter(depth_ls)
print("Counter result:", counter)

# 横軸（depth）と縦軸（Number of CNOTs removed by AQCEL）のデータを生成
depths = list(counter.keys())
counts = list(counter.values())
print("Depths:", depths)
print("Counts:", counts)

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(depths, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Location of CNOT removed by AQCEL')
plt.xlabel('depth')
plt.ylabel('Number of CNOTs removed by AQCEL')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(np.arange(0, max(depths)+1, 10), rotation='vertical')
if max(counts) <= 4:
    plt.yticks(np.arange(0, max(counts)+1, 1))
else:
    plt.yticks(np.arange(0, max(counts)+1, max(counts)//5))

# グラフを保存
plt.savefig('figures/cnot_removed_location.png', bbox_inches='tight')

# グラフを表示
plt.show()


In [ ]:
# 16区間に分けた時の区間ごとのAQCELにより取り除かれたCNOTの数
######################################################################################
##### Number of CNOTs removed by AQCEL per section when divided into 16 sections #####
######################################################################################


# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_ls'の中身はCNOTの消された位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。ただし、区間の端点は整数になるように区切ってください。また、最初の区間の始点は0になるようにしてください。また、最後の区間では最後の値も含むようにしてください。ここではM=16とします
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。m個あればその区間の高さがmとなるような棒グラフを出力してください
# グラフのタイトルは'Number of CNOTs removed by AQCEL per section when divided into 16 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Total number of CNOTs'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。例えば"0-20"のように表示してください。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np

# depth_ls = ...  # データの定義
M = 16

# 最大値を取得し、16区間に分割するための境界を作成
max_val = max(depth_ls)
intervals = np.linspace(0, max_val,M + 1  ).astype(int)
print("Intervals:", intervals)

# 各区間での要素のカウントを保存するための空リストを作成
counts = []

# 各区間の要素をカウント
for i in range(len(intervals)-1):
    count = sum(intervals[i] <= val < intervals[i+1] for val in depth_ls)
    if i == len(intervals)-2:  # 最後の区間では終点も含む
        count += depth_ls.count(intervals[i+1])
    counts.append(count)

print("Counts:", counts)

# 区間ラベルを作成
interval_labels = [f"{intervals[i]}-{intervals[i+1]}" for i in range(len(intervals)-1)]
print("Interval labels:", interval_labels)

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(interval_labels, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Number of CNOTs removed by AQCEL per section when divided into 16 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CNOTs')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(rotation='vertical')
plt.yticks(np.arange(0, max(counts)+1, 5))

# グラフを保存
plt.savefig('figures/cnot_removed_sections.png', bbox_inches='tight')

# グラフを表示
plt.show()



In [ ]:
# 4区間に分けた時の区間ごとのAQCELにより取り除かれたCNOTの数
#####################################################################################
##### Number of CNOTs removed by AQCEL per section when divided into 4 sections #####
#####################################################################################


# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_ls'の中身はCNOTの消された位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。ただし、区間の端点は整数になるように区切ってください。また、最初の区間の始点は0になるようにしてください。また、最後の区間では最後の値も含むようにしてください。ここではM=4とします
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。m個あればその区間の高さがmとなるような棒グラフを出力してください
# グラフのタイトルは'Number of CNOTs removed by AQCEL per section when divided into 4 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Total number of CNOTs'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。例えば"0-20"のように表示してください。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np

# depth_ls = ...  # データの定義
M = 4

# 最大値を取得し、16区間に分割するための境界を作成
max_val = max(depth_ls)
intervals = np.linspace(0, max_val, M +1).astype(int)
print("Intervals:", intervals)

# 各区間での要素のカウントを保存するための空リストを作成
counts = []

# 各区間の要素をカウント
for i in range(len(intervals)-1):
    count = sum(intervals[i] <= val < intervals[i+1] for val in depth_ls)
    if i == len(intervals)-2:  # 最後の区間では終点も含む
        count += depth_ls.count(intervals[i+1])
    counts.append(count)

print("Counts:", counts)

# 区間ラベルを作成
interval_labels = [f"{intervals[i]}-{intervals[i+1]}" for i in range(len(intervals)-1)]
print("Interval labels:", interval_labels)

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(interval_labels, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Number of CNOTs removed by AQCEL per section when divided into 4 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CNOTs')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(rotation='vertical')
plt.yticks(np.arange(0, max(counts)+1, 5))

# グラフを保存
plt.savefig('figures/cnot_removed_sections.png', bbox_inches='tight')

# グラフを表示
plt.show()



In [ ]:
#############################
##### Define depth_cnot #####
#############################

# これより上のメッセージは無視してください
# depth_cnotを同じ値を含む整数を要素に持つリストです。depth_cnotの例を出力してください

# 仮の'depth_cnot'リストを作成します。実際にはこのリストに適切な値を設定してください
depth_cnot = [1, 2, 2, 3, 1, 2, 2, 3, 3, 3, 3, 3, 240]


# depth_cnot = aqcel_results_cc[4]としてください。
depth_cnot = aqcel_results_cc[4]

In [ ]:
#############################################################################
##### Location of CNOT in original circuit (depth) without delimitation #####
#############################################################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_ls'の中身はCNOTの位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸（depth)がnに対してmの高さとなるような棒グラフを出力してください
# グラフのタイトルは'Location of CNOTs removed by AQCEL per section when divided into 16 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Number of CNOTs removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。


import matplotlib.pyplot as plt
from collections import Counter
import numpy as np


# リスト内の各要素の出現回数を数える
counter = Counter(depth_cnot)

# 横軸（depth）と縦軸（Number of CNOTs）のデータを生成
depths = list(counter.keys())
counts = list(counter.values())

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(depths, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Location of CNOT (depth) in original circuit without delimitation')
plt.xlabel('depth')
plt.ylabel('Number of CNOTs')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(np.arange(0, max(depths)+1, 10))
if max(counts) <= 4:
    plt.yticks(np.arange(0, max(counts)+1, 1))
else:
    plt.yticks(np.arange(0, max(counts)+1, max(counts)//5))

# グラフを保存
plt.savefig('figures/cnot_location_depth_count.png')

# グラフを表示
plt.show()





In [ ]:
###########################################
##### Number of CNOTs per 16 segments #####
###########################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cnot'の中身はCNOTの存在する位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。ただし、区間の端点は整数になるように区切ってください。また、最初の区間の始点は0になるようにしてください。また、最後の区間では最後の値も含むようにしてください。ここではM=16とします
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。m個あればその区間の高さがmとなるような棒グラフを出力してください
# グラフのタイトルは'Number of CNOTs per section when divided into 16 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Total number of CNOTs'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。例えば"0-20"のように表示してください。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np

# depth_cnot = aqcel_results_cc[4]

M = 16  # 区間の数

# 深さの最大値を取得し、M個の区間に分割
max_depth = max(depth_cnot)
interval = max_depth // M
bins = [i*interval for i in range(M+1)]
if bins[-1] < max_depth:
    bins[-1] = max_depth  # 最後の区間に最大値も含むようにする

print(f"Max depth: {max_depth}")
print(f"Interval: {interval}")
print(f"Bins: {bins}")

# 各区間でのCNOTの数を計算
cnot_counts = np.histogram(depth_cnot, bins=bins)[0]
print(f"CNOT counts: {cnot_counts}")

# 区間のラベルを作成
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(M)]
print(f"Labels: {labels}")

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(labels, cnot_counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Number of CNOTs per 16 segments')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CNOTs')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(rotation='vertical')  # x軸ラベルが重ならないように縦にする
plt.yticks(np.arange(0, max(cnot_counts)+1, 5))  # y軸の目盛りを0から始めて5の倍数にする

# グラフを保存
plt.tight_layout()  # タイトルやラベルがグラフに被らないようにする
plt.savefig('figures/cnot_counts_per_16_segments.png')

# グラフを表示
plt.show()





In [ ]:
##########################################
##### Number of CNOTs per 4 segments #####
##########################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cnot'の中身はCNOTの存在する位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。ただし、区間の端点は整数になるように区切ってください。また、最初の区間の始点は0になるようにしてください。また、最後の区間では最後の値も含むようにしてください。ここではM=4とします
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。m個あればその区間の高さがmとなるような棒グラフを出力してください
# グラフのタイトルは'Number of CNOTs per section when divided into 4 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Total number of CNOTs'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。例えば"0-20"のように表示してください。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。


import matplotlib.pyplot as plt
import numpy as np

# depth_cnot = aqcel_results_cc[4]

M = 4  # 区間の数

# 深さの最大値を取得し、M個の区間に分割
max_depth = max(depth_cnot)
interval = max_depth // M
bins = [i*interval for i in range(M+1)]
if bins[-1] < max_depth:
    bins[-1] = max_depth  # 最後の区間に最大値も含むようにする

print(f"Max depth: {max_depth}")
print(f"Interval: {interval}")
print(f"Bins: {bins}")

# 各区間でのCNOTの数を計算
cnot_counts = np.histogram(depth_cnot, bins=bins)[0]
print(f"CNOT counts: {cnot_counts}")

# 区間のラベルを作成
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(M)]
print(f"Labels: {labels}")

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(labels, cnot_counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Number of CNOTs per 4 segments')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CNOTs')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(rotation='vertical')  # x軸ラベルが重ならないように縦にする
plt.yticks(np.arange(0, max(cnot_counts)+1, 5))  # y軸の目盛りを0から始めて5の倍数にする

# グラフを保存
plt.tight_layout()  # タイトルやラベルがグラフに被らないようにする
plt.savefig('figures/cnot_counts_per_16_segments.png')

# グラフを表示
plt.show()



In [ ]:
################################################
##### Percentage of CNOTs removed by AQCEL #####
################################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_ls', 'depth_cnot'の中身はそれぞれCNOTの取り除かれた位置、CNOTの位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depth_lsリストのある要素nに対して、nの数をカウントしてください。m個あったとします。同じ値nについて、depth_cnotについて何個あるかカウントしてください。l個とします。あれば横軸（depth)がnに対して高さがm/lとなるような棒グラフを出力してください。ただし、nがdepth_lsに含まれていない場合は高さは0としてください。
# グラフのタイトルは'Percentage of CNOTs removed by AQCEL'
# グラフのx軸は'depth'
# グラフのy軸は'Percentage of CNOTs removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
from collections import Counter

# depth_ls = ...  # データの定義
# depth_cnot = ..

# print(depth_ls)
# print(depth_cnot)
    

# 各要素のカウント
count_depth_ls = Counter(depth_ls)
count_depth_cnot = Counter(depth_cnot)
# print(count_depth_ls)
# print(count_depth_cnot)

# パーセンテージの計算
percentage = {k: count_depth_ls[k] / count_depth_cnot[k] for k in count_depth_cnot.keys()}

print("Percentage:", percentage)

# x軸とy軸の値
x = list(percentage.keys())
y = list(percentage.values())

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(x, y, color='green')

# タイトルと軸のラベルを設定
plt.title('Percentage of CNOTs removed by AQCEL')
plt.xlabel('depth')
plt.ylabel('Percentage of CNOTs removed by AQCEL')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(np.arange(0, max(x)+1, 10), rotation='vertical')
plt.yticks(np.linspace(0, max(y), 5))

# グラフを保存
plt.savefig('figures/percentage_cnot_removed.png', bbox_inches='tight')

# グラフを表示
plt.show()


In [ ]:
# int型のvalueの入ったリスト'depth_ls'について横軸にリストのvalueを。縦軸にそのvalueの表れた回数をプロット
# int型のvalueの入ったリスト'depth_cnot'について横軸にリストのvalueを。縦軸にそのvalueの表れた回数をプロット
# 'depth_ls'と'depth_cnot'について、横軸にリストのvalueを。縦軸に「'depth_ls'のそのvalueの表れた回数 / depth_cnot'のそのvalueの表れた回数をプロット
# 横軸全体をM分割し、M本のヒストグラムによって表示(ここではM = 16とする)
# x軸ラベルは'depth'
# y軸ラベルは'title'




#################################
##### depth of removed CNOT #####
#################################
depth_ls = aqcel_results_cc[2]
print(f'Number of removed CNOTs: {len(depth_ls)}')  #2stepなら122になってほしい #元のcircuitのCNOTが155でAQCEL後には81なので74が出力されれば正解
print(f'Depth of removed CNOTs: {depth_ls}')

############################
##### depth CNOTs exit #####
############################
depth_cnot = aqcel_results_cc[4]
print(f'Number of original CNOTs : {len(depth_cnot)}')#元のcircuitのCNOTの数を表すので155が出力されれば正解
print(f'Depth of original CNOTs : {depth_cnot}')

##########################################
##### graph of depth of removed CNOT #####
##########################################

max_depth =  test.depth()
x = np.arange(max_depth + 1) # 1 to maximum depth
y = np.zeros(max_depth + 1) # initial count is zero
for n in depth_ls:
    y[n] += 1


plt.bar(x, y, color = 'green')
plt.xlim(0.5, max_depth + 0.5)
plt.ylim(0, 5)
#plt.xticks([0, 5, 10], ['c0', 'c6', 'c10])
#plt.yticks([0, 5, 10], ['c0', 'c6', 'c10])
# plt.minorticks_on()
# plt.xscale('log')
# plt.yscale('log')
plt.title('number of the removed CNOTs')
plt.xlabel('depth')
plt.ylabel('number of removed CNOTs')
# plt.grid()
#plt.legend()
#plt.savefig("figures/figure.pdf")
plt.show()

In [ ]:
##########################################
##### graph of depth of removed CNOT #####
##########################################
x = depth_ls


plt.hist(x, bins = 4, color = 'green')
# plt.xlim(0.5, max_depth + 0.5)
# plt.ylim(0, 5)
#plt.xticks([0, 5, 10], ['c0', 'c6', 'c10])
#plt.yticks([0, 5, 10], ['c0', 'c6', 'c10])
# plt.minorticks_on()
# plt.xscale('log')
# plt.yscale('log')
plt.title('number of the removed CNOTs')
plt.xlabel('depth')
plt.ylabel('number of removed CNOTs')
# plt.grid()
#plt.legend()
#plt.savefig("figures/figure.pdf")
plt.show()

In [ ]:
##########################################
##### graph of depth of removed CNOT #####
##########################################

# 64,16
width = 16  # width of histogram

# example:
# >>> np.zeros(5)
# array([ 0.,  0.,  0.,  0.,  0.])

num_cnot_rem = np.zeros(max_depth//width + 1) # array of 6 zeros
for n in range(max_depth): # n = 0 - 5
    if n == (max_depth//width + 1): # max_depth//width + 1 = 256//50 + 1 =  6
        break
    else:
        for m in range(width*n + 1 , width*(n+1) + 1):
            num_cnot_rem[n] += depth_ls.count(m)
            
            
print(f'number of removed CNOTs = {num_cnot_rem}' )
x = np.arange(1, max_depth//width + 2)[:-1] # x = [1 2 3 4 5]
y = num_cnot_rem[:-1]



plt.plot(x, y, color = 'green')
plt.scatter(x, y, marker = 'x', color = 'green')
# plt.xlim(0.5, 4.5)
plt.ylim(0)
# plt.xticks(x , ['1-64', '65-128', '129-192', '195-256'])
# plt.xticks(x , ['1-16', '17-32', '33-48', '49-64', '65-80', '81-96', '97-112', '113-128', '129-144', '145-160', '161-176', '177-192', '193-208', '209-224', '225-240', '240-256'], rotation = 45)
#plt.yticks([0, 5, 10], ['c0', 'c6', 'c10])
# plt.minorticks_on()
# plt.xscale('log')
# plt.yscale('log')
plt.title('number of the removed CNOTs')
plt.xlabel('depth')
plt.ylabel('number of the removed CNOTs')
# plt.grid()
#plt.legend()
#plt.savefig("figures/figure.pdf")
plt.show()

In [ ]:
#################################
##### distribution of CNOTs #####
#################################

# 64,16
width = 16  # width of histogram

# example:
# >>> np.zeros(5)
# array([ 0.,  0.,  0.,  0.,  0.])

num_cnot_ori = np.zeros(max_depth//width + 1) # array of 6 zeros
for n in range(max_depth): # n = 0 - 5
    if n == (max_depth//width + 1): # max_depth//width + 1 = 256//50 + 1 =  6
        break
    else:
        for m in range(width*n + 1 , width*(n+1) + 1):
            num_cnot_ori[n] += depth_cnot.count(m)
            
            
print(f'number of original CNOTs = {num_cnot_ori}')
x = np.arange(1, max_depth//width + 2)[:-1] # x = [1 2 3 4 5]
y = num_cnot_ori[:-1]



plt.plot(x, y, color = 'green')
plt.scatter(x, y, marker = 'x', color = 'green')
# plt.xlim(0.5, 4.5)
plt.ylim(0)
# plt.xticks(x , ['1-64', '65-128', '129-192', '195-256'])
# plt.xticks(x , ['1-16', '17-32', '33-48', '49-64', '65-80', '81-96', '97-112', '113-128', '129-144', '145-160', '161-176', '177-192', '193-208', '209-224', '225-240', '240-256'], rotation = 45)
#plt.yticks([0, 5, 10], ['c0', 'c6', 'c10])
# plt.minorticks_on()
# plt.xscale('log')
# plt.yscale('log')
plt.title('number of CNOTs in original circuit')
plt.xlabel('depth')
plt.ylabel('number of CNOTs in original circuit')
# plt.grid()
#plt.legend()
#plt.savefig("figures/figure.pdf")
plt.show()

In [ ]:
#########################################
##### graph of rate of removed CNOT #####
#########################################

width = 16 # width of histogram

# example:
# >>> np.zeros(5)
# array([ 0.,  0.,  0.,  0.,  0.])

num_cnot_ori = np.zeros(max_depth//width + 1) # array of 6 zeros
num_cnot_rem = np.zeros(max_depth//width + 1) # array of 6 zeros
for n in range(max_depth): # n = 0 - 5
    if n == (max_depth//width + 1): # max_depth//width + 1 = 256//50 + 1 =  6
        break
    else:
        for m in range(width*n + 1 , width*(n+1) + 1):
            num_cnot_ori[n] += depth_cnot.count(m)
            num_cnot_rem[n] += depth_ls.count(m)
            
            
print(f'number of removed CNOTs = {num_cnot_rem}' )
print(f'number of original CNOTs = {num_cnot_ori}')
x = np.arange(1, max_depth//width + 2)[:-1] # x = [1 2 3 4 5]
y = num_cnot_rem[:-1] / num_cnot_ori[:-1]



plt.bar(x, y, color = 'green')
# plt.xlim(0.5, max_depth + 0.5)
# plt.ylim(0, 1)
# plt.xticks(x , ['1-64', '65-128', '129-192', '195-256'])
# plt.xticks(x , ['1-16', '17-32', '33-48', '49-64', '65-80', '81-96', '97-112', '113-128', '129-144', '145-160', '161-176', '177-192', '193-208', '209-224', '225-240', '240-256'], rotation = 45)
#plt.yticks([0, 5, 10], ['c0', 'c6', 'c10])
# plt.minorticks_on()
# plt.xscale('log')
# plt.yscale('log')
plt.title('rate of the removed CNOT')
plt.xlabel('depth')
plt.ylabel('rate of the removed CNOT')
# plt.grid()
#plt.legend()
#plt.savefig("figures/figure.pdf")
plt.show()

In [ ]:
#########################################
##### graph of rate of removed CNOT #####
#########################################

# 64,16
width = 16  # width of histogram

# example:
# >>> np.zeros(5)
# array([ 0.,  0.,  0.,  0.,  0.])

num_cnot_ori = np.zeros(max_depth//width + 1) # array of 6 zeros
num_cnot_rem = np.zeros(max_depth//width + 1) # array of 6 zeros
for n in range(max_depth): # n = 0 - 5
    if n == (max_depth//width + 1): # max_depth//width + 1 = 256//50 + 1 =  6
        break
    else:
        for m in range(width*n + 1 , width*(n+1) + 1):
            num_cnot_ori[n] += depth_cnot.count(m)
            num_cnot_rem[n] += depth_ls.count(m)
            
            
print(f'number of removed CNOTs = {num_cnot_rem}' )
print(f'number of original CNOTs = {num_cnot_ori}')
x = np.arange(1, max_depth//width + 2)[:-1] # x = [1 2 3 4 5]
y = num_cnot_rem[:-1] / num_cnot_ori[:-1]



plt.plot(x, y, color = 'green')
plt.scatter(x, y, marker = 'x', color = 'green')
# plt.xlim(0.5, 4.5)
plt.ylim(0, 1)
# plt.xticks(x , ['1-64', '65-128', '129-192', '195-256'])
# plt.xticks(x , ['1-16', '17-32', '33-48', '49-64', '65-80', '81-96', '97-112', '113-128', '129-144', '145-160', '161-176', '177-192', '193-208', '209-224', '225-240', '240-256'], rotation = 45)
#plt.yticks([0, 5, 10], ['c0', 'c6', 'c10])
# plt.minorticks_on()
# plt.xscale('log')
# plt.yscale('log')
plt.title('rate of the removed CNOT')
plt.xlabel('depth')
plt.ylabel('rate of the removed CNOT')
# plt.grid()
#plt.legend()
#plt.savefig("figures/figure.pdf")
plt.show()

In [ ]:
##############################################
##### depth of removed Toffoli into CNOT #####
##############################################
depth_ls_tof = aqcel_results_cc[3]
print(depth_ls_tof)

In [ ]:
#######################################################
##### graph of depth of removed Toffoli into CNOT #####
#######################################################
##########################################
##### graph of depth of removed CNOT #####
##########################################
max_depth =  test.depth()
x = np.arange(max_depth + 1) # 1 to maximum depth
y = np.zeros(max_depth + 1) # initial count is zero
for n in depth_ls_tof:
    y[n] += 1


plt.bar(x, y)
plt.xlim(0.5, max_depth + 0.5)
plt.ylim(0, 3)
#plt.xticks([0, 5, 10], ['c0', 'c6', 'c10])
#plt.yticks([0, 5, 10], ['c0', 'c6', 'c10])
# plt.minorticks_on()
# plt.xscale('log')
# plt.yscale('log')
plt.title('depth of the removed CNOT')
plt.xlabel('depth')
plt.ylabel('the number of removed CNOTs')
# plt.grid()
#plt.legend()
#plt.savefig("figures/figure.pdf")
plt.show()

In [ ]:
############################
##### draw the circuit #####
############################
aqcel_results_cc[0].draw('mpl', fold=100)

In [ ]:
#####################################
##### count the number of CNOTs #####
#####################################
aqcel_results_cc[0].count_ops()

#### ④AQCEL circuit optimization decomposed

In [ ]:
#################################
##### decompose the circuit #####
#################################
aqcel_results_cc_2 = aqcel_results_cc[0].decompose()
aqcel_results_cc_2.draw('mpl',fold=100)

In [ ]:
#####################################
##### count the number of CNOTs #####
#####################################
aqcel_results_cc_2.count_ops()